# Rolex Listing Price Prediction based on model and complications

This model aims to predict the fair price of any Rolex watches that are listed on [Chrono24.com](https://www.chrono24.ca/), in the hope of determining whether a piece of Rolex is a "good deal". The best optimized model has an $R^2$ score of 0.655. 

In [1]:
import pandas as pd
import numpy as np
import glob
import janitor
import altair as alt
import matplotlib as plt
alt.data_transformers.enable("vegafusion")
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
import warnings
warnings.filterwarnings("ignore")

# imports
import sys, os
import time

import numpy as np
import pandas as pd
import altair as alt
from IPython.display import HTML

sys.path.append(os.path.join(os.path.abspath("."), "code"))

from IPython.display import display

# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# Preprocessing and pipeline
from sklearn.impute import SimpleImputer

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    RobustScaler
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.svm import *
from lightgbm.sklearn import *
from sklearn.model_selection import *
from xgboost import XGBRegressor

In [2]:
# Read the cleaned data
rolex_df = pd.read_csv('data/rolex_df.csv')
train_df, test_df = train_test_split(rolex_df, test_size=0.2, random_state=123)
print(train_df.shape)
print(test_df.shape)

(49996, 21)
(12499, 21)


In [3]:
X_train, y_train = train_df.drop(
    columns=["price"]), train_df["price"]
y_train = pd.DataFrame(y_train)
X_test, y_test = test_df.drop(
    columns=["price"]), test_df["price"]
y_test = pd.DataFrame(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(49996, 20)
(49996, 1)
(12499, 20)
(12499, 1)


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49996 entries, 698 to 52734
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   model                 49996 non-null  object 
 1   movement              49159 non-null  object 
 2   case_material         48678 non-null  object 
 3   bracelet_material     45462 non-null  object 
 4   year_of_production    37355 non-null  float64
 5   year_is_approximated  49996 non-null  int64  
 6   condition             49234 non-null  object 
 7   scope_of_delivery     49996 non-null  object 
 8   country               49996 non-null  object 
 9   availability          49996 non-null  object 
 10  case_diameter         49996 non-null  int64  
 11  bezel_material        36838 non-null  object 
 12  crystal               40790 non-null  object 
 13  dial                  46103 non-null  object 
 14  bracelet_color        38567 non-null  object 
 15  clasp                 

## Models

### Preprocessing

In [5]:
# Function to quicly cross validate different models
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, n_jobs=-1, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" %
                       (mean_scores.iloc[i], std_scores.iloc[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

One-hot encoding is applied on categorical features and scaling on numerical features.

In [6]:
categorcial_feats = [col for col in X_train.columns if col not in ['case_diameter', 'rating', 'reviews', 'year_of_production']]
numerical_feats = ['case_diameter', 'rating', 'reviews', 'year_of_production']

categorical_pipe = make_pipeline(OneHotEncoder(drop='if_binary', handle_unknown='ignore'))
numerical_pipe = make_pipeline(SimpleImputer(strategy='median'), RobustScaler())

preprocessor_with_scaler = make_column_transformer((categorical_pipe, categorcial_feats),
                                                    (numerical_pipe, numerical_feats))
preprocessor_with_scaler

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['model', 'movement', 'case_material',
                                  'bracelet_material', 'year_is_approximated',
                                  'condition', 'scope_of_delivery', 'country',
                                  'availability', 'bezel_material', 'crystal',
                                  'dial', 'bracelet_color', 'clasp',
                                  'clasp_material', 'is_negotiable']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['case_diameter', 'rating', 'reviews',
                                  'year_of_production'])])

### Model Fitting

In [7]:
# create a dictionary for storing model scores
results_dict = {}

#### Baseline - Simple Linear Regression

In [8]:
linear_reg = make_pipeline(preprocessor_with_scaler,
                           LinearRegression(n_jobs=-1))
results_dict["linear regression"] = mean_std_cross_val_scores(
    linear_reg, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)


#### Classical Linear Regression Models: Ridge and Lasso

In [9]:
ridge = make_pipeline(preprocessor_with_scaler,
                      Ridge())
results_dict["ridge"] = mean_std_cross_val_scores(
    ridge, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)


In [10]:
lasso = make_pipeline(preprocessor_with_scaler,
                      Lasso())
results_dict["lasso"] = mean_std_cross_val_scores(
    lasso, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)


In [11]:
elasticnet = make_pipeline(preprocessor_with_scaler,
                           ElasticNet())
results_dict["elastic net"] = mean_std_cross_val_scores(
    elasticnet, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)
elastic net,0.806 (+/- 0.039),0.107 (+/- 0.016),0.243 (+/- 0.034),0.239 (+/- 0.008)


The linear regression models are not performing well as expected, but they will serve as our baseline models.

#### Tree-based Models

In [12]:
dt = make_pipeline(preprocessor_with_scaler, DecisionTreeRegressor())
results_dict["decision tree"] = mean_std_cross_val_scores(
    dt, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)
elastic net,0.806 (+/- 0.039),0.107 (+/- 0.016),0.243 (+/- 0.034),0.239 (+/- 0.008)
decision tree,8.529 (+/- 0.169),0.100 (+/- 0.006),0.361 (+/- 0.219),0.994 (+/- 0.001)


In [13]:
rf = make_pipeline(preprocessor_with_scaler, RandomForestRegressor(random_state=123))
results_dict["random forest"] = mean_std_cross_val_scores(
    rf, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)
elastic net,0.806 (+/- 0.039),0.107 (+/- 0.016),0.243 (+/- 0.034),0.239 (+/- 0.008)
decision tree,8.529 (+/- 0.169),0.100 (+/- 0.006),0.361 (+/- 0.219),0.994 (+/- 0.001)
random forest,545.417 (+/- 1.992),0.455 (+/- 0.026),0.634 (+/- 0.041),0.946 (+/- 0.002)


In [14]:
xgboost = make_pipeline(preprocessor_with_scaler, XGBRegressor(objective='reg:gamma', # so that the predictions would be non-negative
                                                               random_state=123,
                                                               n_jobs=-1,
                                                               verbosity=0,))
results_dict["xgboost"] = mean_std_cross_val_scores(
    xgboost, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)
elastic net,0.806 (+/- 0.039),0.107 (+/- 0.016),0.243 (+/- 0.034),0.239 (+/- 0.008)
decision tree,8.529 (+/- 0.169),0.100 (+/- 0.006),0.361 (+/- 0.219),0.994 (+/- 0.001)
random forest,545.417 (+/- 1.992),0.455 (+/- 0.026),0.634 (+/- 0.041),0.946 (+/- 0.002)
xgboost,1.558 (+/- 0.064),0.146 (+/- 0.011),0.596 (+/- 0.038),0.711 (+/- 0.007)


Even though the Random Forest outperformed our XGBoost model, it took a lot more time to train, which could be detrimental to the subsequent hyperparameter tuning.

#### Distance-based Models

In [15]:
knn = make_pipeline(preprocessor_with_scaler, KNeighborsRegressor())
results_dict["knn"] = mean_std_cross_val_scores(
    knn, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)
elastic net,0.806 (+/- 0.039),0.107 (+/- 0.016),0.243 (+/- 0.034),0.239 (+/- 0.008)
decision tree,8.529 (+/- 0.169),0.100 (+/- 0.006),0.361 (+/- 0.219),0.994 (+/- 0.001)
random forest,545.417 (+/- 1.992),0.455 (+/- 0.026),0.634 (+/- 0.041),0.946 (+/- 0.002)
xgboost,1.558 (+/- 0.064),0.146 (+/- 0.011),0.596 (+/- 0.038),0.711 (+/- 0.007)
knn,0.494 (+/- 0.011),64.874 (+/- 0.485),0.508 (+/- 0.019),0.689 (+/- 0.006)


Distance-based models are performing decently, but the fit time is still long. 

Given that hyperparameters would need to be tuned, having a short train time would greatly reduce the time needed. We will proceed with XGBoost for now.

### Hyperparameter Optimization

The hyperparameters are tuned on a GCP virtual machine (32-core with 32GB RAM).

In [16]:
TUNING = False

During the first 2 rounds of tuning, it was noticed that the 2 regularization hyperparameters `gamma` and `alpha` have clear linear relationship with the validation score. Therefore they are removed from the `param_grid` subsequently to increase the chance of hitting relevant hyperparameter values.

In [17]:
param_grid = {
    "xgbregressor__learning_rate": np.arange(0.1, 1, 0.05),
    "xgbregressor__max_depth": np.arange(6, 15, 1),
    "xgbregressor__max_leaves": np.arange(300, 2001, 50),
    "xgbregressor__n_estimators": np.arange(100, 2001, 50),
    # "xgbregressor__gamma": np.arange(0, 100, 0.5),
    "xgbregressor__lambda": np.arange(0, 100, 0.5),
    # "xgbregressor__alpha": np.arange(0, 100, 0.5)

}

Each of the tuning lasted around 100 minutes.

In [18]:
if TUNING:
    random_search = RandomizedSearchCV(
    xgboost,
    param_distributions=param_grid,
    n_iter=500,
    n_jobs=-1,
    return_train_score=True,
    random_state=123
    )

    random_search.fit(X_train, y_train)

In [19]:
if TUNING:
    cv_result_df = pd.DataFrame(random_search.cv_results_)[
        [
            "mean_test_score",
            "param_xgbregressor__learning_rate",
            "param_xgbregressor__max_depth",
            "param_xgbregressor__max_leaves",
            "param_xgbregressor__n_estimators",
            # "param_xgbregressor__gamma",
            "param_xgbregressor__lambda",
            # "param_xgbregressor__alpha",
            "mean_fit_time",
            "rank_test_score",
        ]
    ].set_index("rank_test_score").sort_index().T

    cv_result_df.to_csv('model/xgboost_cv_result.csv')
    cv_result_df
else:
    cv_result_df = pd.read_csv('model/xgboost_cv_result.csv', index_col=0)

cv_result_df

,1,2,3,4,5,6,7,8,9,10,...,491,492,493,494,495,496,497,498,499,500
mean_test_score,0.666200,0.666004,0.665954,0.664258,0.664003,0.663649,0.663486,0.662884,0.662490,0.661591,...,0.598779,0.590099,0.586650,0.580672,0.576403,0.570624,0.566720,0.561730,0.558923,0.113361
param_xgbregressor__learning_rate,0.300000,0.450000,0.400000,0.300000,0.250000,0.200000,0.400000,0.600000,0.200000,0.400000,...,0.900000,0.650000,0.100000,0.500000,0.200000,0.400000,0.100000,0.200000,0.200000,0.100000
param_xgbregressor__max_depth,18.000000,15.000000,12.000000,16.000000,16.000000,9.000000,18.000000,8.000000,9.000000,11.000000,...,9.000000,6.000000,7.000000,7.000000,10.000000,6.000000,7.000000,6.000000,7.000000,18.000000
param_xgbregressor__max_leaves,800.000000,950.000000,800.000000,500.000000,1750.000000,900.000000,450.000000,750.000000,1350.000000,950.000000,...,1200.000000,400.000000,500.000000,1100.000000,1750.000000,1650.000000,900.000000,1400.000000,550.000000,800.000000
param_xgbregressor__n_estimators,800.000000,1550.000000,750.000000,1850.000000,800.000000,1200.000000,450.000000,1100.000000,900.000000,650.000000,...,450.000000,100.000000,750.000000,100.000000,100.000000,150.000000,350.000000,200.000000,150.000000,100.000000
param_xgbregressor__lambda,97.500000,76.000000,77.500000,99.000000,79.500000,24.500000,58.500000,87.000000,11.000000,89.000000,...,0.500000,48.000000,77.000000,68.500000,67.500000,70.000000,59.500000,55.000000,81.500000,30.500000
mean_fit_time,61.327716,111.045194,27.815104,97.699582,52.892310,28.034589,22.899016,24.115249,19.262149,22.036837,...,13.688438,2.078513,12.182348,2.645134,3.125023,3.098966,5.282631,3.388408,2.931280,2.530644


In [20]:
fig_hyperparam = alt.Chart(cv_result_df.T).mark_point(clip=True).encode(
    x = alt.X('mean_test_score').scale(domain=(0.58, 0.7)),
    y=alt.Y(alt.repeat("row"), type='quantitative')
).properties(
    width=900
).repeat(
    row=cv_result_df.T.columns.to_list()[1:-1]
)

fig_hyperparam

alt.RepeatChart(...)

It appears that a learning rate below 0.5 and a higher `lambda` result in higher validation score, even though it is not too significant.  
  
The model could be further tuned and improved, if time and resources permit.

In [21]:
import pickle

if TUNING:
    xgboost_opt = make_pipeline(random_search.best_estimator_)
else:
    with open('model/xgboost_opt.pkl', 'rb') as file:
        xgboost_opt = pickle.load(file) 
        display(xgboost_opt)
                            
results_dict["xgboost optimized"] = mean_std_cross_val_scores(
    xgboost_opt, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['model', 'movement',
                                                   'case_material',
                                                   'bracelet_material',
                                                   'year_is_approximated',
                                                   'condition',
                                                   'scope_of_delivery',
                                                   'country', 'availability',
                                                   'bezel_material', 'crystal',
                                                   'dial', 'brac...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, lambda=97.5,
                              learning_rate=0.30000000000000004, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=18, max_leaves=800,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=800, n_jobs=-1,
                              num_parallel_tree=None, ...))])

,fit_time,score_time,test_score,train_score
linear regression,3.112 (+/- 0.113),0.118 (+/- 0.010),0.440 (+/- 0.037),0.444 (+/- 0.011)
ridge,2.091 (+/- 0.027),0.095 (+/- 0.005),0.440 (+/- 0.038),0.444 (+/- 0.011)
lasso,22.720 (+/- 0.173),0.096 (+/- 0.012),0.440 (+/- 0.038),0.444 (+/- 0.011)
elastic net,0.806 (+/- 0.039),0.107 (+/- 0.016),0.243 (+/- 0.034),0.239 (+/- 0.008)
decision tree,8.529 (+/- 0.169),0.100 (+/- 0.006),0.361 (+/- 0.219),0.994 (+/- 0.001)
random forest,545.417 (+/- 1.992),0.455 (+/- 0.026),0.634 (+/- 0.041),0.946 (+/- 0.002)
xgboost,1.558 (+/- 0.064),0.146 (+/- 0.011),0.596 (+/- 0.038),0.711 (+/- 0.007)
knn,0.494 (+/- 0.011),64.874 (+/- 0.485),0.508 (+/- 0.019),0.689 (+/- 0.006)
xgboost optimized,36.151 (+/- 1.957),7.013 (+/- 1.503),0.655 (+/- 0.036),0.980 (+/- 0.002)


In [22]:
# Fit the optimized model
xgboost_opt.fit(X_train, y_train)

if TUNING:
    # Save the model to a pickle file
    with open('model/xgboost_opt.pkl', 'wb') as file:
        pickle.dump(xgboost_opt, file)

A more interpretable metric:

In [23]:
from sklearn.metrics import mean_absolute_percentage_error

print(f'Each prediction is expected to have a {(mean_absolute_percentage_error(y_train["price"], xgboost_opt.predict(X_train))*100):.3f}% error.')

prediction = y_train.copy()
prediction['pred'] = xgboost_opt.predict(X_train)
prediction['residual'] = prediction['price'] - prediction['pred']
prediction['perc_error'] = prediction['residual'] / prediction['price']

display(prediction.sort_values('perc_error').head(10))
display(prediction.sort_values('perc_error').tail(10))

Each prediction is expected to have a 5.260% error.


,price,pred,residual,perc_error
52423,466,3088.498779,-2622.498779,-5.627680
42417,75549,262857.468750,-187308.468750,-2.479298
36831,2322,6935.457520,-4613.457520,-1.986846
22723,38208,102287.859375,-64079.859375,-1.677132
23174,15283,39359.550781,-24076.550781,-1.575381
52445,198,483.905914,-285.905914,-1.443969
40989,52662,128261.265625,-75599.265625,-1.435556
54112,26433,63272.996094,-36839.996094,-1.393712
5084,3317,7695.925293,-4378.925293,-1.320146
26811,24735,55308.558594,-30573.558594,-1.236044


,price,pred,residual,perc_error
41872,52874,26584.248047,26289.751953,0.497215
53879,155269,77963.359375,77305.640625,0.497882
23017,72498,35713.261719,36784.738281,0.507390
55106,29603,14239.364258,15363.635742,0.518989
58703,63627,29949.878906,33677.121094,0.529290
57770,83476,38682.210938,44793.789062,0.536607
40925,302959,130178.875000,172780.125000,0.570309
38851,254719,102287.859375,152431.140625,0.598429
41099,143944,55308.558594,88635.441406,0.615763
55213,27754,10061.434570,17692.565430,0.637478


The model is not quite good at predicting fair prices of watch pieces that are too expensive or too cheap.

In [24]:
alt.Chart(prediction,
          title='Histogram of predicted price').mark_bar().encode(
    alt.X('pred:Q').bin(maxbins=60),
    y='count()'
)

alt.Chart(...)

### Prediction on Test Set

The score on the test set is close to the validation score obtained from the train set, which is a good sign of generalization.

In [25]:
xgboost_opt.score(X_test, y_test)

0.6349789595037301

In [26]:
test_pred = y_test.copy()
test_pred['prediction'] = xgboost_opt.predict(X_test)
test_pred['residual'] = test_pred['price'] - test_pred['prediction']
test_pred.head()

,price,prediction,residual
25189,81660,96680.406250,-15020.406250
43476,23558,27019.271484,-3461.271484
17555,11972,10489.150391,1482.849609
31938,19316,15398.788086,3917.211914
42483,60300,80784.359375,-20484.359375


In [27]:
prediction_plot = alt.Chart(test_pred, title='Actual Listing Price and Prediction').mark_point().encode(
    alt.X('price').title('Actual Price'),
    alt.Y('prediction').title('Predicted Price')
)

min_price = min(test_pred['price'].min(), test_pred['prediction'].min())
max_price = max(test_pred['price'].max(), test_pred['prediction'].max())

# Create a DataFrame for the 45-degree line
line_data = pd.DataFrame({
    'price': [min_price, max_price],
    'prediction': [min_price, max_price]
})

# Create the 45-degree line chart
line_chart = alt.Chart(line_data).mark_line(color='red').encode(
    x='price',
    y='prediction'
)

prediction_plot + line_chart

alt.LayerChart(...)

Zooming into the CA$120,000 zone:

In [28]:
fig_pred = alt.Chart(test_pred).mark_point(clip=True).encode(
    x=alt.X('price').scale(domain=[0, 120000]),
    y=alt.Y('prediction').scale(domain=[0, 120000])
)
line_chart = alt.Chart(line_data).mark_line(color='red', clip=True).encode(
    x=alt.X('price').scale(domain=[0, 120000]),
    y=alt.Y('prediction').scale(domain=[0, 120000])
)

fig_pred 
(fig_pred + line_chart).configure_mark(
    opacity=0.3
)

alt.LayerChart(...)